# Exploring Background Noise

This notebook aims to analyze an unedited vinyl rip in `.wav` format. It will:

First we mess with some parameters of the plotting package so graphs appear nicely and import some useful libraries.

In [1]:
import itertools

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [40, 8]

import numpy as np
np.set_printoptions(suppress=True)

from scipy.io import wavfile

Import the audio analysis libraries

In [2]:
from pyAudioAnalysis import audioBasicIO as aIO
from pyAudioAnalysis import audioSegmentation as aS

We import a .wav file and take the first track to analyze. 

In [3]:
filename = 'media/A.wav'
[sampling_rate, signal] = aIO.read_audio_file(filename)
signal = aIO.stereo_to_mono(signal)
print(f"Track is sampled at {sampling_rate}Hz with {signal.shape[0]} samples.")

Track is sampled at 44100Hz with 47815168 samples.


Now we try to split it into tracks. On this record, we expect it to find 7 tracks with boundaries at roughly
1. 2-123
2. 127-242
3. 246-460
4. 465-632
5. 634-766
6. 769-924
7. 930-1078

In [ ]:
segments = aS.silence_removal(signal, sampling_rate, 0.020, 0.020, smooth_window = 2, weight = 0.05, plot = True)

In [ ]:
segments

A priori, we know tracks are have a couple seconds between them, so we join any segments which don't have more than a second between them.

In [ ]:
filter_short = list(itertools.chain(*segments.copy()))

for end, start in zip(filter_short[1::2], filter_short[2::2]):
    if start - end < 1:
        filter_short.remove(end)
        filter_short.remove(start)

filter_short = [[start, end] for start, end in zip(filter_short[::2], filter_short[1::2])]
filter_short

After this we try to fix fragmentation some more by sticking chunks less than 45 seconds long to their closest neighbor.

In [ ]:
def dur(pair):
    return pair[1] - pair[0]

combine_short = filter_short.copy()

pair = min(combine_short, key=dur)
while(dur(pair) < 45):
    ix = combine_short.index(pair)
    
    if ix == 0:
        left_merge = False
    elif ix == len(combine_short) - 1:
        left_merge = True
    else:
        left_merge = combine_short[ix][0] - combine_short[ix-1][1] < combine_short[ix+1][0] - combine_short[ix][1]
        
    if left_merge:
        combine_short[ix-1][1] = combine_short[ix][1]
    else:
        combine_short[ix+1][0] = combine_short[ix][0]
    
    combine_short.pop(ix)

    pair = min(combine_short, key=dur)

combine_short

This correlates pretty well to the tracking. We write our 'silent' intervals to `.wav` for inspection.

In [ ]:
def seconds_to_samples(intervals, sampling_rate):
    return [[int(start * sampling_rate), int(end * sampling_rate)] for start, end in intervals]

def invert_intervals(intervals, n_samples):
    inverted = list(itertools.chain(*intervals.copy()))
    if inverted[0] == 0:
        inverted.pop(0)
    else:
        inverted.insert(0, 0)
        
    if inverted[-1] == n_samples-1:
        inverted.pop(-1)
    else:
        inverted.append(n_samples-1)
        
    return [[start, end] for start, end in zip(inverted[::2], inverted[1::2])]

In [ ]:
silent_samples_intervals = invert_intervals(seconds_to_samples(combine_short, sampling_rate), signal.shape[0])

ixs = list(itertools.chain(*[range(start, end) for start, end in silent_samples_intervals]))
wavfile.write(f"silence.wav", sampling_rate, signal[ixs].astype('int16'))

Although it relatively accurately describes the position of track changes, it carries too much noise to effectively estimate silent volume.